In [1]:
from cuda import cuda, cudart, nvrtc
import numpy as np

def ASSERT_DRV(err):
    if isinstance(err, cuda.CUresult):
        if err != cuda.CUresult.CUDA_SUCCESS:
            raise RuntimeError("Cuda Error: {}".format(err))
    elif isinstance(err, nvrtc.nvrtcResult):
        if err != nvrtc.nvrtcResult.NVRTC_SUCCESS:
            raise RuntimeError("Nvrtc Error: {}".format(err))
    else:
        raise RuntimeError("Unknown error type: {}".format(err))

In [2]:
AAAAA =  """\

#include "helper_math.h"

#define cl_device __device__

cl_device float sign( float x )
{
  return x > 0.0 ? 1.0 : (x < 0.0 ? -1.0 : 0.0);
}


cl_device float dot2( float3 x )
{
  return dot(x, x);
}

cl_device float sdf_triangle( float3 p, float3 *a, float3 *b, float3 *c )
{
  float3 ba = *b - *a; float3 pa = p - *a;
  float3 cb = *c - *b; float3 pb = p - *b;
  float3 ac = *a - *c; float3 pc = p - *c;
  float3 nor = cross( ba, ac );

  return sqrt(
    (sign(dot(cross(ba,nor),pa)) +
     sign(dot(cross(cb,nor),pb)) +
     sign(dot(cross(ac,nor),pc))<2.0)
     ?
     min( min(
     dot2(ba*clamp(dot(ba,pa)/dot2(ba),0.0,1.0)-pa),
     dot2(cb*clamp(dot(cb,pb)/dot2(cb),0.0,1.0)-pb) ),
     dot2(ac*clamp(dot(ac,pc)/dot2(ac),0.0,1.0)-pc) )
     :
     dot(nor,pa)*dot(nor,pa)/dot2(nor) );
}


cl_device float get_sign( float3 p, float3 a, float3 b, float3 c )
{
  float3 ba = b - a; 
  float3 ac = a - c; 
  float3 nor = cross( ba, ac );

  return sign(dot(p-a, nor));
}
    
    
extern "C" 
{ 



__global__ void voxelize(
    float *vert,
    int facenum,
    float *mesh)
{

    uint3 id = blockDim*blockIdx + threadIdx;
    uint3 dim = make_uint3(256, 128, 256);//blockDim*gridDim;
    
    float3 voxel = (make_float3(id.x, id.y, id.z) - make_float3(128,0,128)) 
                                                                              * make_float3(1000,500,1000) / make_float3(256,128,256);

    int idx = 0;
    float dist = 1e+9;

    for (int i = 0; i < facenum; ++i) {
        int n = i*9;

        //int3 fid = *(int3*)(&face[n]) * 3;
        //if ( ((*(float3*)(&vert[fid.x])).y + (*(float3*)(&vert[fid.y])).y + (*(float3*)(&vert[fid.z])).y) < 10.0) continue;
        //float sh = length(*(float3*)(&vert[n])- voxel);
        //if (dist < sh) continue;

        
        
        float t = sdf_triangle(voxel, 
            (float3*)(&vert[n]), 
            (float3*)(&vert[n+3]),
            (float3*)(&vert[n+6])
        );

        if (t < dist) {
            dist = t;
            idx = n;
        }
    }



    //int3 fid = *(int3*)(&face[idx]) * 3;

    (mesh)[id.x + id.y*dim.x + id.z*dim.x *dim.y] = dist ;  
    

    
    /* get_sign(voxel,
                                                                  *(float3*)(&vert[idx]), 
                                                                  *(float3*)(&vert[idx+3]),
                                                                  *(float3*)(&vert[idx+6])
                                                                            );
                                                                            */
}
} // extern C
"""

In [3]:

def _cudaGetErrorEnum(error):
    if isinstance(error, cuda.CUresult):
        err, name = cuda.cuGetErrorName(error)
        return name if err == cuda.CUresult.CUDA_SUCCESS else "<unknown>"
    elif isinstance(error, cudart.cudaError_t):
        return cudart.cudaGetErrorName(error)[1]
    elif isinstance(error, nvrtc.nvrtcResult):
        return nvrtc.nvrtcGetErrorString(error)[1]
    else:
        raise RuntimeError('Unknown error type: {}'.format(error))

def checkCudaErrors(result):
    if result[0].value:
        raise RuntimeError("CUDA error code={}({})".format(result[0].value, _cudaGetErrorEnum(result[0])))
    if len(result) == 1:
        return None
    elif len(result) == 2:
        return result[1]
    else:
        return result[1:]

In [4]:
# import os

# CUDA_HOME = os.getenv('CUDA_HOME')
# if CUDA_HOME == None:
#     CUDA_HOME = os.getenv('CUDA_PATH')
# if CUDA_HOME == None:
#     raise RuntimeError('Environment variable CUDA_HOME or CUDA_PATH is not set')
# include_dirs = os.path.join(CUDA_HOME, 'include')




# Create program
err, prog = nvrtc.nvrtcCreateProgram(str.encode(AAAAA), b"sdfpy.cu", 0, [], [])

# Compile program 
# cuda_runtime.h 라는 파일에 경로   /usr/local/cuda-1xxxxxxx/  이하 경로 어딘가  
PATH = '/usr/local/cuda-12.1/targets/x86_64-linux/include'

opts = ["--include-path={} ".format(PATH).encode('UTF-8')]
err, = nvrtc.nvrtcCompileProgram(prog, len(opts), opts)

# compile err log
logSize = checkCudaErrors(nvrtc.nvrtcGetProgramLogSize(prog))
log = b' ' * logSize
checkCudaErrors(nvrtc.nvrtcGetProgramLog(prog, log))
print(log.decode())
ASSERT_DRV(err)

# Get PTX from compilation
err, ptxSize = nvrtc.nvrtcGetPTXSize(prog)
ASSERT_DRV(err)
ptx = b" " * ptxSize
err, = nvrtc.nvrtcGetPTX(prog, ptx)
ASSERT_DRV(err)

sdfpy.cu(65): warning #550-D: variable "idx" was set but never used
      int idx = 0;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

 


In [5]:
# Initialize CUDA Driver API
err, = cuda.cuInit(0)
ASSERT_DRV(err)
# Retrieve handle for device 0
err, cuDevice = cuda.cuDeviceGet(0)
ASSERT_DRV(err)
# Create context
err, context = cuda.cuCtxCreate(0, cuDevice)
ASSERT_DRV(err)


# Load PTX as module data and retrieve function
ptx = np.char.array(ptx)
# Note: Incompatible --gpu-architecture would be detected here
err, module = cuda.cuModuleLoadData(ptx.ctypes.data)
ASSERT_DRV(err)
err, kernel = cuda.cuModuleGetFunction(module, b"voxelize")
ASSERT_DRV(err)

In [6]:
import trimesh  
from openfoam.prep import *


X = 256
Y = 128
Z = 256
SIZE = X*Y*Z


polymesh = parse_stl_file('./refCaseSolved3/constant/triSurface/background3.stl')


pointss = list(map(lambda x: x.points, polymesh))
points = np.array(pointss).ravel()

hbuv = np.array(points+[0.0]).astype(dtype=np.float32)

hbulen = np.array([(len(points)-1)/9]).astype(dtype=np.int32)

hOut = np.zeros(SIZE, dtype=np.float32)



floatsize = np.int32(4)






#  gpu 메모리 할당

err, dVclass = cuda.cuMemAlloc(len(hbuv)*floatsize)
ASSERT_DRV(err)

err, dOutclass = cuda.cuMemAlloc(SIZE * floatsize)
ASSERT_DRV(err)


err, stream = cuda.cuStreamCreate(0)
ASSERT_DRV(err)


# 할당한 메모리로 복사
ASSERT_DRV(err)
err, = cuda.cuMemcpyHtoDAsync(
   dVclass, hbuv.ctypes.data, len(hbuv)*floatsize, stream
)
ASSERT_DRV(err)
err, = cuda.cuMemcpyHtoDAsync(
   dOutclass, hOut.ctypes.data, SIZE * floatsize, stream
)
ASSERT_DRV(err)



# 인자로 넣을 준비

# The following code example is not intuitive 
# Subject to change in a future release
dY = np.array([int(dVclass)], dtype=np.uint64)


dOut = np.array([int(dOutclass)], dtype=np.uint64)

args = [dY, hbulen, dOut]
args = np.array([arg.ctypes.data for arg in args], dtype=np.uint64)

In [7]:
err, = cuda.cuLaunchKernel(
   kernel,
   X/8,  # grid x dim
   Y/8,  # grid y dim
   Z/8,  # grid z dim
   8,  # block x dim
   8,  # block y dim
   8,  # block z dim
   0,  # dynamic shared memory
   stream,  # stream
   args.ctypes.data,  # kernel arguments
   0,  # extra (ignore)
)
ASSERT_DRV(err)

err, = cuda.cuCtxSynchronize()
ASSERT_DRV(err)

In [8]:
err, = cuda.cuMemcpyDtoHAsync(
   hOut.ctypes.data, dOutclass, SIZE *4, stream
)
ASSERT_DRV(err)

err, = cuda.cuStreamSynchronize(stream)
ASSERT_DRV(err)


# 인자로 슨 메모리 해제   오브젝트마다 크기다르기에 재사용 ㄴ
err, = cuda.cuMemFree(dVclass)
err, = cuda.cuMemFree(dOutclass)

In [9]:

err, = cuda.cuStreamDestroy(stream)


err, = cuda.cuModuleUnload(module)
err, = cuda.cuCtxDestroy(context)

In [1]:
from get_sdf import *

hOut = get_sdfs(['./refCaseSolved3/constant/triSurface/background3.stl'])


hOut.reshape((256,128,256))

sdfpy.cu(50): warning #550-D: variable "idx" was set but never used
      int idx = 0;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

 


array([[[9.99999885e-03, 9.99999978e-03, 9.99999978e-03, ...,
         9.99999978e-03, 9.99999978e-03, 9.99999978e-03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[9.99999978e-03, 9.99999978e-03, 9.99999978e-03, ...,
         9.99999978e-03, 9.99999978e-03, 9.99999978e-03],
        [0.00000000e+00, 3.89625025e+00, 3.89625025e+00, ...,
         3.89625025e+00, 3.89625025e+00, 3.89625001e+00],
        [0.00000000e+00, 

In [2]:
import cv2



cv2.namedWindow('v xy',cv2.WINDOW_NORMAL)
# cv2.namedWindow('ref v xy',cv2.WINDOW_NORMAL)
# image = a.cpu().detach().numpy()[3][:, 5, :]/4
# rimage = ref[0].cpu().detach().numpy().reshape(5,16,16,16)[3][:, 5, :]/4

# print(rimage.shape)
# print(rimage)
# #image = cv2.cvtColor(image,cv2.COLOR_HSV2BGR)
# cv2.imshow('ref v xy',rimage)
cv2.imshow('v xy', hOut.reshape((256,128,256))[:,30,:] / 256.)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
# import plotly.graph_objects as go

# Xi, Yi, Zi = np.mgrid[-1.5:1.5:256j, 0:3:128j,-1.5:1.5:256j]

# fig = go.Figure(data=go.Volume(
#     x=Xi.flatten(),
#     y=Yi.flatten(),
#     z=Zi.flatten(),
#     value=hOut,
#     isomin=0.0,
#     isomax=300.8,
#     opacity=0.1, # needs to be small to see through all surfaces
#     surface_count=256, # needs to be a large number for good volume rendering
#     ))
# fig.show()

In [ ]:
with open('kern.cu', 'rt') as cuda_file:
    cuda_source = cuda_file.read()

err, prog = nvrtc.nvrtcCreateProgram(str.encode(cuda_source), b"kernel.cu", 0, [], [])

In [ ]:
face_type = np.dtype([('v0', (gpuarray.vec.float4,32)), ('v1', (gpuarray.vec.float4,32)), ('v2', (gpuarray.vec.float4,32))])

polymesh = save_and_rename_bounding_box()

tri_face = np.empty(0 , dtype=face_type)

for po in polymesh:
    v0 = gpuarray.vec.make_float4(po.points[0][0], po.points[0][1], po.points[0][2], 0.0)
    v1 = gpuarray.vec.make_float4(po.points[1][0], po.points[1][1], po.points[1][2], 0.0)
    v2 = gpuarray.vec.make_float4(po.points[2][0], po.points[2][1], po.points[2][2], 0.0)
    tri_face.append([v0, v1, v2])



tri_face_gpu = cuda.mem_alloc(tri_face.nbytes)

output_mesh = np.empty((X*Y*Z), np.float32)
output_mesh_gpu = cuda.mem_alloc(output_mesh.nbytes)


with open('kern.cu', 'rt') as cuda_file:
    cuda_source = SourceModule(cuda_file.read(), include_dirs=['./'], no_extern_c=True)


cuda.memcpy_htod(tri_face_gpu, tri_face)


func_V = cuda_source.get_function("voxelize")

func_V(tri_face_gpu, np.int32(len(tri_face)), output_mesh_gpu,  block=(8,8,8), grid=(X, Y, Z))

cuda.Context.synchronize()


cuda.memcpy_dtoh(output_mesh, output_mesh_gpu)



